In [ ]:
import numpy as np
import pandas as pd
import joblib

# Documentation Header
"""
### Load and Use the Trained Model for Predictions

This script loads the trained model and makes predictions on new data.

Steps:
1. Load the trained model from file.
2. Load the new dataset.
3. Preprocess the dataset (ensure it matches the training format).
4. Make predictions using the trained model.
5. Save the predictions if necessary.
"""

# Load the trained model
model_path = "./src/models/best_model.joblib"
model = joblib.load(model_path)
print(f"Model loaded from {model_path}")

# Load new data (replace with actual file path)
new_data_path = "./src/data/new_data.csv"
df_new = pd.read_csv(new_data_path)

# Ensure the new data does not contain the target column
if "INCOME" in df_new.columns:
    df_new = df_new.drop(columns=["INCOME"])

# Make predictions
predictions = model.predict(df_new)

# Add predictions to the dataframe
df_new["Predicted_INCOME"] = predictions

# Save the predictions
output_path = "./src/data/predictions.csv"
df_new.to_csv(output_path, index=False)
print(f"Predictions saved to {output_path}")
